In [1]:
import pandas as pd
import statsmodels.api as sm # 회귀모델
import joblib # joblib로 모델 저장/모델 load

In [3]:
# 자동완성 핫키 : ctrl+space
df = pd.read_csv('data/apt_api.csv', comment='#', encoding='cp949')
df.sample()

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,해제사유발생일,해제여부
47,10500,2013,2021,숭인동,종로아인스빌,7,5,12.15,1392-1,11110,15,-,-


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   거래금액     318 non-null    int64  
 1   건축년도     318 non-null    int64  
 2   년        318 non-null    int64  
 3   법정동      318 non-null    object 
 4   아파트      318 non-null    object 
 5   월        318 non-null    int64  
 6   일        318 non-null    int64  
 7   전용면적     318 non-null    float64
 8   지번       318 non-null    object 
 9   지역코드     318 non-null    int64  
 10  층        318 non-null    int64  
 11  해제사유발생일  318 non-null    object 
 12  해제여부     318 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 32.4+ KB


In [12]:
# pd.options.mode.copy_on_write = True
X = df[['건축년도', '전용면적','층']].copy()
X['const'] = 1
y = df['거래금액']
X.shape, y.shape

((318, 4), (318,))

In [ ]:
# 회귀모델